In [1]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from itertools import islice, takewhile, chain
from functools import reduce
from typing import Optional
import datetime as dt
from dataclasses import asdict, fields
from importlib import reload

from geopy.distance import distance
from shapely.geometry import Point, LineString
import shapely.geometry as sg
import geopandas as gpd

from typing import List
import ipyleaflet as lf

pd.set_option('display.max_rows', 1000000000)

In [2]:
import busboy.model as m
import busboy.geo as geo
import busboy.database as db
import busboy.prediction as prediction
import busboy.map.map as bmap
import busboy.apis as api
import busboy.util as util
import busboy.util.notebooks as notebook

/Users/Noel/.local/share/virtualenvs/Busboy-8t8akAoa/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [39]:
reload(util)
reload(geo)
reload(m)
reload(db)
reload(prediction)
reload(bmap)
reload(api)
reload(notebook)

<module 'busboy.util.notebooks' from '/Users/Noel/Developer/Projects/Busboy/busboy/util/notebooks.py'>

In [3]:
rbn = db.routes_by_name()
route = rbn["220"].id
entries = db.snapshots(r=route, d=dt.date(2019, 2, 18))
stops_by_name = db.stops_by_name()
timetables = api.timetables("220", stops_by_name)
timetable_variants = {t for timetable in timetables for t in timetable.variants}

In [4]:
entries_by_vehicle = util.dict_collect_list(entries, lambda e: e.vehicle)
vehicles_by_entry_count = [v for (v, es) in sorted(entries_by_vehicle.items(), key = lambda t: len(t[1]), reverse=True)]

In [5]:
pvars = sorted(list(
    prediction.possible_variants(
        prediction.drop_duplicate_positions(entries_by_vehicle[vehicles_by_entry_count[1]]), 
        timetable_variants
    )),
    key = lambda t: t[0].poll_time)

In [6]:
order_pvars = list(prediction.check_variant_order(pvars))

In [84]:
precheck = [(len(t[1]), t[0].latitude, t[0].longitude, t[0].poll_time.time().isoformat(), {(tv.route, i, tv.stops[i].name, tv.stops[i + 1].name) for (tv, i) in t[1]}) for t in islice(pvars, 200, 300)]
precheck

[(6,
  51.87828416666667,
  -8.436893055555556,
  '20:01:27.593951',
  {('220',
    12,
    'Douglas Village East (Shopping Centre)',
    'Douglas Road (South Link Exit Slip Road)'),
   ('220',
    18,
    'Douglas Village East (Shopping Centre)',
    'Douglas Road (South Link Exit Slip Road)'),
   ('220',
    21,
    'Douglas Village East (Shopping Centre)',
    'Douglas Road (South Link Exit Slip Road)'),
   ('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (6,
  51.87699805555555,
  -8.435628888888889,
  '20:01:47.597543',
  {('220',
    11,
    'Maryborough Hill (Paddocks)',
    'Douglas Village East (Shopping Centre)'),
   ('220',
    17,
    'Maryborough Hill (Paddocks)',
    'Douglas Village East (Shopping Centre)'),
   ('220',
    20,
    'Maryborough Hill (Paddocks)',
    'Douglas Village East (Shopping Centre)'),
   ('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (6,
  51.8

In [85]:
postcheck = [(len(t[1]), t[0].latitude, t[0].longitude, t[0].poll_time.time().isoformat(), {(tv.route, i, tv.stops[i].name, tv.stops[i + 1].name) for (tv, i) in t[1]}) for t in islice(order_pvars, 200, 300)]
postcheck

[(3,
  51.87828416666667,
  -8.436893055555556,
  '20:01:27.593951',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:01:47.597543',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:02:07.608235',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:02:27.624700',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:02:47.641593',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East Village (Opp Tramway Tce)')}),
 (3,
  51.87699805555555,
  -8.435628888888889,
  '20:03:07.658397',
  {('220',
    43,
    'Douglas Road (Clermont Ave)',
    'Douglas East

In [40]:
route_sections = {v: list(prediction.route_sections(v.stops)) for v in timetable_variants}
stop_shaped_entries = [(e, {v: {t[1] for t in ts} for v, ts in util.dict_collect_set(vs, lambda tpl: tpl[0]).items()}) for (e, vs) in order_pvars]
times = prediction.stop_times(stop_shaped_entries, route_sections)
section_times = prediction.section_times(stop_shaped_entries, route_sections)
journeys = prediction.journeys(section_times)

In [164]:
positions = [(s.poll_time, ps) for s, ps in stop_shaped_entries]
positions

[(datetime.datetime(2019, 2, 18, 8, 16, 14, 859009),
  {(route: 220, start: Ovens (Grange Road Terminus), end: Fort Camden): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Carrigaline (Forrest Hills)): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Crosshaven (Village Ctr Northbound)): {0}}),
 (datetime.datetime(2019, 2, 18, 8, 22, 35, 74702),
  {(route: 220, start: Ovens (Grange Road Terminus), end: Fort Camden): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Carrigaline (Forrest Hills)): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Crosshaven (Village Ctr Northbound)): {0}}),
 (datetime.datetime(2019, 2, 18, 8, 25, 35, 130512),
  {(route: 220, start: Ovens (Grange Road Terminus), end: Fort Camden): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Carrigaline (Forrest Hills)): {0},
   (route: 220, start: Ovens (Grange Road Terminus), end: Crosshaven (Village Ctr Northbound)): {0}}),
 (datetime.datetime(201

In [178]:
js = list(journeys.values())[0]
journey = js[0]
journey

[(0,
  (Nothing(), Just(value=datetime.datetime(2019, 2, 18, 8, 16, 14, 859009))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 25, 35, 130512)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)))),
 (1,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 25, 35, 130512)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473)))),
 (2,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 28, 15, 238443)))),
 (3,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270)),
   Just(value=datetime.datetime(201

In [182]:
padded = prediction.pad_journeys(journeys)
pjs = list(padded.values())[0]
pjourney = pjs[0]
pjourney

[(0,
  (Nothing(), Just(value=datetime.datetime(2019, 2, 18, 8, 16, 14, 859009))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 25, 35, 130512)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)))),
 (1,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 25, 35, 130512)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473)))),
 (2,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 28, 15, 238443)))),
 (3,
  (Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)),
   Just(value=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473))),
  (Just(value=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270)),
   Just(value=datetime.datetime(201

In [165]:
[len(t) for v, ts in times.items() for t in ts]

[17633, 16064, 17497, 16706, 15302, 12582]

In [168]:
for (variant, these_times) in section_times.items():
    print(f"{variant}:")
    for j, t in enumerate(these_times):
        p, (t1, t2), (t3, t4) = t
        print(f"Section {p} (index {j})")
        print(f"Last before: {t1.map(lambda t: t.time().isoformat())}")
        print(f"First in: {t2.map(lambda t: t.time().isoformat())}")
        print(f"Last in: {t3.map(lambda t: t.time().isoformat())}")
        print(f"First after: {t4.map(lambda t: t.time().isoformat())}")

(route: 220, start: Ovens (Grange Road Terminus), end: Crosshaven (Village Ctr Northbound)):
Section 0 (index 0)
Last before: Nothing()
First in: Just(value='08:16:14.859009')
Last in: Just(value='08:25:35.130512')
First after: Just(value='08:26:15.164377')
Section 1 (index 1)
Last before: Just(value='08:25:35.130512')
First in: Just(value='08:26:15.164377')
Last in: Just(value='08:26:15.164377')
First after: Just(value='08:26:35.181473')
Section 2 (index 2)
Last before: Just(value='08:26:15.164377')
First in: Just(value='08:26:35.181473')
Last in: Just(value='08:27:35.216270')
First after: Just(value='08:28:15.238443')
Section 3 (index 3)
Last before: Just(value='08:26:15.164377')
First in: Just(value='08:26:35.181473')
Last in: Just(value='08:27:35.216270')
First after: Just(value='08:28:15.238443')
Section 4 (index 4)
Last before: Just(value='08:27:35.216270')
First in: Just(value='08:28:15.238443')
Last in: Just(value='08:28:15.238443')
First after: Just(value='08:28:35.244090')
Se

Last before: Just(value='19:43:39.825246')
First in: Just(value='19:43:59.837787')
Last in: Just(value='19:43:59.837787')
First after: Just(value='19:44:19.854441')
Section 85 (index 356)
Last before: Just(value='19:43:59.837787')
First in: Just(value='19:44:19.854441')
Last in: Just(value='19:45:59.922702')
First after: Just(value='19:46:19.939486')
Section 86 (index 357)
Last before: Just(value='19:45:59.922702')
First in: Just(value='19:46:19.939486')
Last in: Just(value='19:46:19.939486')
First after: Just(value='19:46:59.966535')
Section 87 (index 358)
Last before: Just(value='19:46:19.939486')
First in: Just(value='19:46:59.966535')
Last in: Just(value='19:47:39.992099')
First after: Just(value='19:48:00.002507')
Section 88 (index 359)
Last before: Just(value='19:47:39.992099')
First in: Just(value='19:48:00.002507')
Last in: Just(value='19:48:00.002507')
First after: Just(value='19:48:20.019121')
Section 90 (index 360)
Last before: Just(value='19:48:00.002507')
First in: Just(va

First after: Just(value='12:27:04.642766')
Section 33 (index 116)
Last before: Just(value='12:26:24.613639')
First in: Just(value='12:27:04.642766')
Last in: Just(value='12:27:44.673623')
First after: Just(value='12:28:04.688749')
Section 34 (index 117)
Last before: Just(value='12:27:44.673623')
First in: Just(value='12:28:04.688749')
Last in: Just(value='12:28:44.718940')
First after: Just(value='12:29:04.727962')
Section 35 (index 118)
Last before: Just(value='12:28:44.718940')
First in: Just(value='12:29:04.727962')
Last in: Just(value='12:30:04.778302')
First after: Just(value='12:30:24.793430')
Section 37 (index 119)
Last before: Just(value='12:30:04.778302')
First in: Just(value='12:30:24.793430')
Last in: Just(value='12:30:24.793430')
First after: Just(value='12:30:44.809192')
Section 38 (index 120)
Last before: Just(value='12:30:24.793430')
First in: Just(value='12:30:44.809192')
Last in: Just(value='12:30:44.809192')
First after: Just(value='12:31:04.824251')
Section 40 (index

First after: Just(value='22:44:46.932014')
Section 70 (index 416)
Last before: Just(value='22:44:26.914545')
First in: Just(value='22:44:46.932014')
Last in: Just(value='22:45:06.939405')
First after: Just(value='22:45:26.954633')
Section 71 (index 417)
Last before: Just(value='22:45:06.939405')
First in: Just(value='22:45:26.954633')
Last in: Just(value='22:45:46.971579')
First after: Just(value='22:46:06.988784')
Section 72 (index 418)
Last before: Just(value='22:45:46.971579')
First in: Just(value='22:46:06.988784')
Last in: Just(value='22:46:06.988784')
First after: Just(value='22:46:27.004051')
Section 73 (index 419)
Last before: Just(value='22:46:06.988784')
First in: Just(value='22:46:27.004051')
Last in: Just(value='22:46:27.004051')
First after: Just(value='22:46:47.010569')
Section 74 (index 420)
Last before: Just(value='22:46:27.004051')
First in: Just(value='22:46:47.010569')
Last in: Just(value='22:46:47.010569')
First after: Just(value='22:47:07.027483')
Section 76 (index

First in: Just(value='15:46:11.261639')
Last in: Just(value='15:47:31.290266')
First after: Just(value='15:47:51.306071')
Section 8 (index 201)
Last before: Just(value='15:47:31.290266')
First in: Just(value='15:47:51.306071')
Last in: Just(value='15:47:51.306071')
First after: Just(value='15:48:11.322130')
Section 9 (index 202)
Last before: Just(value='15:47:51.306071')
First in: Just(value='15:48:11.322130')
Last in: Just(value='15:48:11.322130')
First after: Just(value='15:48:31.337738')
Section 11 (index 203)
Last before: Just(value='15:48:11.322130')
First in: Just(value='15:48:31.337738')
Last in: Just(value='15:49:11.368225')
First after: Just(value='15:49:31.383911')
Section 13 (index 204)
Last before: Just(value='15:49:11.368225')
First in: Just(value='15:49:31.383911')
Last in: Just(value='15:49:31.383911')
First after: Just(value='15:50:11.410173')
Section 15 (index 205)
Last before: Just(value='15:49:31.383911')
First in: Just(value='15:50:11.410173')
Last in: Just(value='1

...

Section 7 (index 263)
Last before: Just(value='20:49:02.402461')
First in: Just(value='20:49:22.418461')
Last in: Just(value='20:49:22.418461')
First after: Just(value='20:50:02.446276')
Section 8 (index 264)
Last before: Just(value='20:49:22.418461')
First in: Just(value='20:50:02.446276')
Last in: Just(value='20:50:02.446276')
First after: Just(value='20:50:22.447342')
Section 9 (index 265)
Last before: Just(value='20:50:02.446276')
First in: Just(value='20:50:22.447342')
Last in: Just(value='20:50:22.447342')
First after: Just(value='20:50:42.448444')
Section 11 (index 266)
Last before: Just(value='20:50:22.447342')
First in: Just(value='20:50:42.448444')
Last in: Just(value='20:50:42.448444')
First after: Just(value='20:51:22.456764')
Section 13 (index 267)
Last before: Just(value='20:50:42.448444')
First in: Just(value='20:51:22.456764')
Last in: Just(value='20:51:22.456764')
First after: Just(value='20:51:42.461263')
Section 14 (index 268)
Last before: Just(value='20:51:22.456764

Section 116 (index 463)
Last before: Just(value='02:24:08.166854')
First in: Just(value='02:24:28.174692')
Last in: Just(value='02:24:28.174692')
First after: Just(value='02:24:48.189633')
Section 117 (index 464)
Last before: Just(value='02:24:28.174692')
First in: Just(value='02:24:48.189633')
Last in: Just(value='02:25:08.193065')
First after: Just(value='02:26:48.214763')
Section 118 (index 465)
Last before: Just(value='02:25:08.193065')
First in: Just(value='02:26:48.214763')
Last in: Just(value='02:26:48.214763')
First after: Just(value='02:27:08.230456')
Section 119 (index 466)
Last before: Just(value='02:26:48.214763')
First in: Just(value='02:27:08.230456')
Last in: Just(value='02:27:28.249872')
First after: Just(value='02:28:08.288572')
Section 120 (index 467)
Last before: Just(value='02:27:28.249872')
First in: Just(value='02:28:08.288572')
Last in: Just(value='02:28:08.288572')
First after: Just(value='02:28:28.303471')
(route: 220, start: Crosshaven (Village Ctr Northbound)

Last in: Just(value='17:48:55.461547')
First after: Just(value='17:49:15.474449')
Section 39 (index 195)
Last before: Just(value='17:48:55.461547')
First in: Just(value='17:49:15.474449')
Last in: Just(value='17:49:15.474449')
First after: Just(value='17:49:55.505460')
Section 40 (index 196)
Last before: Just(value='17:49:15.474449')
First in: Just(value='17:49:55.505460')
Last in: Just(value='17:50:15.521071')
First after: Just(value='17:50:35.536529')
Section 42 (index 197)
Last before: Just(value='17:50:15.521071')
First in: Just(value='17:50:35.536529')
Last in: Just(value='17:50:55.551309')
First after: Just(value='17:51:15.565238')
Section 44 (index 198)
Last before: Just(value='17:50:55.551309')
First in: Just(value='17:51:15.565238')
Last in: Just(value='17:51:15.565238')
First after: Just(value='17:51:35.578834')
Section 46 (index 199)
Last before: Just(value='17:51:15.565238')
First in: Just(value='17:51:35.578834')
Last in: Just(value='17:51:35.578834')
First after: Just(val

First in: Just(value='15:03:29.993164')
Last in: Just(value='15:05:50.037803')
First after: Just(value='15:06:10.052884')
Section 75 (index 133)
Last before: Just(value='15:05:50.037803')
First in: Just(value='15:06:10.052884')
Last in: Just(value='15:07:30.115908')
First after: Just(value='15:07:50.120402')
Section 76 (index 134)
Last before: Just(value='15:07:30.115908')
First in: Just(value='15:07:50.120402')
Last in: Just(value='15:07:50.120402')
First after: Just(value='15:08:30.125547')
Section 77 (index 135)
Last before: Just(value='15:07:50.120402')
First in: Just(value='15:08:30.125547')
Last in: Just(value='15:08:30.125547')
First after: Just(value='15:08:50.129264')
Section 78 (index 136)
Last before: Just(value='15:08:30.125547')
First in: Just(value='15:08:50.129264')
Last in: Just(value='15:08:50.129264')
First after: Just(value='15:09:10.133434')
Section 79 (index 137)
Last before: Just(value='15:08:50.129264')
First in: Just(value='15:09:10.133434')
Last in: Just(value=

Last before: Just(value='23:54:42.066659')
First in: Just(value='23:55:22.100690')
Last in: Just(value='23:55:42.119358')
First after: Just(value='23:56:22.156337')
Section 63 (index 333)
Last before: Just(value='23:55:42.119358')
First in: Just(value='23:56:22.156337')
Last in: Just(value='23:57:02.186519')
First after: Just(value='23:57:22.205043')
Section 64 (index 334)
Last before: Just(value='23:57:02.186519')
First in: Just(value='23:57:22.205043')
Last in: Just(value='23:57:22.205043')
First after: Just(value='23:57:42.223679')
Section 66 (index 335)
Last before: Just(value='23:57:22.205043')
First in: Just(value='23:57:42.223679')
Last in: Just(value='23:58:02.242102')
First after: Just(value='23:58:02.242102')
Section 68 (index 336)
Last before: Just(value='23:58:02.242102')
First in: Just(value='23:58:02.242102')
Last in: Just(value='23:58:22.249174')
First after: Just(value='23:58:42.252473')
Section 70 (index 337)
Last before: Just(value='23:58:22.249174')
First in: Just(va

In [74]:
first_snapshot = stop_shaped_entries[40]
positions = util.first(first_snapshot[1].values()).value
positions
[(v.stops[0].name, ts) for s in stop_shaped_entries for v, ts in s[1].items()]

[('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange Road Terminus)', {66}),
 ('Ovens (Grange

In [42]:
for variant, these_stop_times in times.items():
    print(f"Variant: {variant}")
    print(f"{len(these_stop_times)} journeys")
    for trip_number, trip_times in enumerate(these_stop_times):
        for stop_number, stop_time in enumerate(trip_times):
            to_time = lambda d: d.time().isoformat()
            print(f"- {trip_number}, {stop_number:2}, {variant.stops[stop_number].name:40}")
            print(f"{stop_time}")

Variant: (route: 220, start: Ovens (Grange Road Terminus), end: Fort Camden)
5 journeys
- 0,  0, Ovens (Grange Road Terminus)            
SeenAtStop(last_before=Nothing(), first_at=datetime.datetime(2019, 2, 18, 8, 16, 14, 859009), last_at=datetime.datetime(2019, 2, 18, 8, 25, 35, 130512), last_after=Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)))
- 0,  1, Ovens (Opp Grange Manor)                
SeenAtStop(last_before=Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)), first_at=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473), last_at=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270), last_after=Just(value=datetime.datetime(2019, 2, 18, 8, 28, 15, 238443)))
- 0,  2, Killumney Road (Kilumney Cross)         
SeenAtStop(last_before=Just(value=datetime.datetime(2019, 2, 18, 8, 26, 15, 164377)), first_at=datetime.datetime(2019, 2, 18, 8, 26, 35, 181473), last_at=datetime.datetime(2019, 2, 18, 8, 27, 35, 216270), last_after=Just(value=datetime.datetime(2019, 2, 18

- 2, 32, Sheares Street (Mercy Hospital)         
SeenAtStop(last_before=Just(value=datetime.datetime(2019, 2, 18, 16, 26, 32, 781309)), first_at=datetime.datetime(2019, 2, 18, 16, 26, 52, 782939), last_at=datetime.datetime(2019, 2, 18, 16, 28, 32, 831701), last_after=Just(value=datetime.datetime(2019, 2, 18, 16, 28, 52, 834603)))
- 2, 33, Grand Parade (Caseys Furniture)         
SeenAtStop(last_before=Just(value=datetime.datetime(2019, 2, 18, 16, 30, 32, 870392)), first_at=datetime.datetime(2019, 2, 18, 16, 31, 12, 890130), last_at=datetime.datetime(2019, 2, 18, 16, 33, 52, 987107), last_after=Just(value=datetime.datetime(2019, 2, 18, 16, 34, 13, 2245)))
- 2, 34, South Mall (Opp Cork Passport Office)   
SeenAtStop(last_before=Just(value=datetime.datetime(2019, 2, 18, 16, 34, 33, 17739)), first_at=datetime.datetime(2019, 2, 18, 16, 34, 53, 33197), last_at=datetime.datetime(2019, 2, 18, 16, 36, 53, 102117), last_after=Just(value=datetime.datetime(2019, 2, 18, 16, 37, 13, 117145)))
- 2, 

- 4, 53, Ballincollig West (White Horse Bar)     
SeenAtStop(last_before=Just(value=datetime.datetime(2019, 2, 19, 0, 8, 42, 624570)), first_at=datetime.datetime(2019, 2, 19, 0, 9, 2, 643400), last_at=datetime.datetime(2019, 2, 19, 0, 9, 2, 643400), last_after=Just(value=datetime.datetime(2019, 2, 19, 0, 9, 22, 661224)))
- 4, 54, Ballincollig West (Coolroe Heights)     
SeenAtStop(last_before=Just(value=datetime.datetime(2019, 2, 19, 0, 9, 2, 643400)), first_at=datetime.datetime(2019, 2, 19, 0, 9, 22, 661224), last_at=datetime.datetime(2019, 2, 19, 0, 9, 22, 661224), last_after=Just(value=datetime.datetime(2019, 2, 19, 0, 10, 2, 690376)))
- 4, 55, Ballincollig West (Old Quarry)          
SeenAtStop(last_before=Just(value=datetime.datetime(2019, 2, 19, 0, 9, 22, 661224)), first_at=datetime.datetime(2019, 2, 19, 0, 10, 2, 690376), last_at=datetime.datetime(2019, 2, 19, 0, 10, 2, 690376), last_after=Just(value=datetime.datetime(2019, 2, 19, 0, 10, 22, 708585)))
- 4, 56, Ballincollig West 

In [117]:
calculated_times = set()

for v, ts in islice(times.items(), 0, 10):
    for p, stop_times in islice(ts.items(), 0, 100):
        for t in stop_times:
            calculated_times.add(tuple(chain([p, v.stops[p].name], (x for x in t))))
            
            
for (position, name, t1, t2) in sorted(calculated_times, key=lambda tpl: tpl[2]):
    print(f"{position:3}: {name:50} ({t1.time().isoformat()} -> {t2.time().isoformat()})")

AttributeError: 'list' object has no attribute 'items'

In [40]:
themap = bmap.Map()
themap.map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [43]:
timetables208 = list(api.timetables("208", stops_by_name))
timetables205 = list(api.timetables("205", stops_by_name))

In [46]:
notebook.show_timetables(themap, timetables208)

In [49]:
notebook.show_timetables(themap, timetables205)

In [30]:
notebook.plot_entries(
    themap, 
    (t[0] for t in islice(stop_shaped_entries, 400, 600)), 
    snapshot_to_layer = lambda e: lf.Marker(
        location=(e.latitude, e.longitude),
        draggable=False,
        title=e.poll_time.time().isoformat()
    ),
    clear=False
)

In [7]:
vehicles_by_entry_count

[VehicleId(raw='7338674957838188756'),
 VehicleId(raw='7338674957838188752'),
 VehicleId(raw='7338674957838188925'),
 VehicleId(raw='7338674957838188758'),
 VehicleId(raw='7338674957838188929'),
 VehicleId(raw='7338674957838188928'),
 VehicleId(raw='7338674957838189413'),
 VehicleId(raw='7338674957838188953'),
 VehicleId(raw='7338674957838188804'),
 VehicleId(raw='7338674957838188930'),
 VehicleId(raw='7338674957838189374'),
 VehicleId(raw='7338674957838188809'),
 VehicleId(raw='7338674957838188917'),
 VehicleId(raw='7338674957838188920'),
 VehicleId(raw='7338674957838188879'),
 VehicleId(raw='7338674957838189409'),
 VehicleId(raw='7338674957838189406'),
 VehicleId(raw='7338674957838188838'),
 VehicleId(raw='7338674957838189385'),
 VehicleId(raw='7338674957838189370'),
 VehicleId(raw='7338674957838188805'),
 VehicleId(raw='7338674957838188762'),
 VehicleId(raw='7338674957838188973'),
 VehicleId(raw='7338674957838188927'),
 VehicleId(raw='7338674957838188918'),
 VehicleId(raw='733867495